# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [4]:
# import required libraries
import numpy as np
import pandas as pd
import sqlalchemy

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [5]:
# your code here
conn_string = 'mysql+pymysql://root:&FK^[8jLK$"/4=n*@34.65.10.136:3306/orders'
conn = sqlalchemy.create_engine(conn_string)
orders = pd.read_sql_query('SELECT * FROM orders;', conn)

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '34.65.10.136' (timed out)")
(Background on this error at: http://sqlalche.me/e/e3q8)

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [7]:
#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?
orders.drop('index',axis=1)
ord_gb = orders.groupby('CustomerID').sum()
ord_gb = ord_gb['amount_spent'].sort_values(ascending=False)
ord_gb = pd.DataFrame(ord_gb)

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?
ord_gb['Percentile_rank'] = ord_gb.amount_spent.rank(pct=True)
vip_cust = ord_gb[(ord_gb['Percentile_rank']>=0.95)]
pref_cust = ord_gb[(ord_gb['Percentile_rank']>=0.75) & (ord_gb['Percentile_rank']<=0.95)]

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?
def f(row):
    if row['Percentile_rank'] >= 0.95:
        val = 'VIP'
    elif row['Percentile_rank'] >= 0.75 and row['Percentile_rank'] < 0.95:
        val = 'Preferred'
    else:
        val = 'Regular'
    return val

ord_gb['cust_label'] = ord_gb.apply(f, axis=1)
ord_gb

NameError: name 'orders' is not defined

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [3]:
cc_gb = orders.groupby(['CustomerID','Country']).sum()
cc_gb.reset_index(inplace=True)
cust_df = ord_gb.merge(cc_gb[['CustomerID','Country']],on='CustomerID',how='left')
cust_df

#identifying the country with most VIP customers
top_vip = cust_df.groupby('CustomerID','cust_label').count()

NameError: name 'orders' is not defined

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [ ]:
# your code here
top_vip_pref = cust_df.groupby('CustomerID','cust_label').count()